<a href="https://colab.research.google.com/github/sandhrabijoy/Pytorch-and-ML-learning/blob/master/MCQmodel_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers torch nltk sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━ 196.6/363.4 MB 25.9 MB/s eta 0:00:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 15.1 MB/s eta 0:00:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may

In [2]:
import random
import nltk
import torch
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util

In [3]:
nlk = nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
question_generator=pipeline('text2text-generation',model='google-bert/bert-base-uncased')
qa_pipeline=pipeline('question-answering',model='distilbert-base-cased-distilled-squad')
similarity_model=SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
def generate_mcqs(text, num_questions=3):
    # Tokenize the text into sentences
    sentences = nltk.sent_tokenize(text)
    mcqs = []

    for _ in range(num_questions):
        # Step 1: Select a random sentence
        sentence = random.choice(sentences)

        # Step 2: Generate a question from the selected sentence
        question_input = f"highlight: {sentence} </s>"
        question = question_generator(question_input, max_length=64, do_sample=False)[0]["generated_text"]

        # Step 3: Extract the answer using the QA model
        answer = qa_pipeline(question=question, context=sentence)["answer"]

        # Step 4: Generate distractors (related options) from other sentences
        options = [answer]  # Start with the correct answer

        # Create distractors from other sentences (using related questions approach)
        for s in sentences[:3]:  # Using the first 3 sentences as context for distractors
            distractor = qa_pipeline(question="What can be the related question to the given sentence?", context=s)["answer"]
            if distractor and distractor != answer:  # Avoid duplicates and empty distractors
                options.append(distractor)

        # Step 5: Clean up options
        options = list(set(filter(None, options)))  # Remove None values and duplicates
        options = [opt.capitalize()[:50] for opt in options]  # Capitalize first letter and limit to 50 characters
        random.shuffle(options)  # Shuffle the options randomly

        # Step 6: Append the MCQ question, choices, and correct answer
        mcqs.append({
            'question': question,
            'choices': options,
            'answer': answer
        })

    return mcqs




In [6]:
text_input=input("\nEnter the context:\n")


Enter the context:
 The Adventures of Roderick Random, Scottish author and poet Tobias Smollett’s debut picaresque novel, was published in 1748 and is loosely based on Smollett’s experiences in the British Navy. Set in the 1730s and 1740s, the narrative follows the life of the main protagonist and narrator, Roderick “Rory” Random, from childhood to adulthood as Roderick journeys through England, France, the Caribbean, Africa, and Latin America. The Adventures of Roderick Random’s synopsis details the book as “one of the first truly global novels [that] casts light on nearly every aspect of its time—imperialism, gender relations, slavery, urban life, colonial warfare, commerce, politics, the professions, high society, and the Hogarthian underworld.”  Roderick’s father is a Scottish gentleman, but his mother is a common woman. His father’s high-class family is ashamed of the son’s marriage to a woman from low society and subsequently shuns Roderick’s parents. Roderick’s mother dies soon

In [7]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [8]:
mcq_questions=generate_mcqs(text_input)

In [9]:
for idx, mcq in enumerate(mcq_questions, 1):
    print(f"\nQ{idx}: {mcq['question']}")
    for i, choice in enumerate(mcq['choices'], 1):
        print(f"   {i}. {choice}")
    print(f"Answer: {mcq['answer']}\n")


Q1: highlight : roderick ’ s schoolmasters are no better and constantly use him as a whipping boy and a model for punishment, even when he ’ s done nothing wrong. < / s > | | | | | | | | | | | | | for him, even he and roderick are equal. | |
   1. Journeys through england, france, the caribbean, a
   2. His mother is a common woman
   3. Constantly use him as a whipping boy and a model f
   4. Experiences in the british navy
Answer: constantly use him as a whipping boy and a model for punishment


Q2: highlight : because he ’ s a sailor in the navy, bowling isn ’ t always present for roderick. < / s > < > < > < > < > < > < > bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling bowling
   1. His mother is a common woman
   2. Experiences in the british navy
   3. Journeys through england, france, the caribbean, a
   4. Roderick
Answer: Roderick


Q3